In [1]:
from __future__ import print_function
import keras
import pandas as pd
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from time import time
from sklearn.metrics import roc_auc_score as rocauc
from sklearn.metrics import accuracy_score as acc
from sklearn.model_selection import train_test_split
from __future__ import print_function  
import seaborn as sns
sns.set_style("whitegrid")
np.set_printoptions(precision=10)
np.set_printoptions(suppress=True)
input_shape = (28, 28, 1)

Using TensorFlow backend.


In [2]:
def D1_problem_to_C1_problem(arr, fractions):
    number_of_C0_in_D0, number_of_C0_in_D1, number_of_C1_in_D0, number_of_C1_in_D1 = fractions
    alpha = number_of_C0_in_D0 / (number_of_C0_in_D0 + number_of_C1_in_D0)
    beta = number_of_C0_in_D1 / (number_of_C0_in_D1 + number_of_C1_in_D1)
    arr = (arr - beta/(alpha + beta))*(((1 - beta) / (2 - alpha - beta) - beta / (alpha + beta)) ** -1)
    arr = np.clip(arr,0,1) # значение недообученного классификатора на предыдущем этапе может выйти за границы 0 и 1 
    return arr

In [3]:
def generate_dataset(C0_values, fractions):
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x = np.append(x_train, x_test, axis=0)
    y = np.append(y_train, y_test)
    x = x.reshape(x.shape[0], 28, 28, 1)
    y_C1 = np.array([int(elem not in C0_values) for elem in y])

    NUMBER_OF_C0_IN_D0, NUMBER_OF_C0_IN_D1, NUMBER_OF_C1_IN_D0, NUMBER_OF_C1_IN_D1 = fractions
    i, X, Y_C1, Y_D1 = 0, [], [], []

    while (NUMBER_OF_C0_IN_D0, NUMBER_OF_C0_IN_D1, NUMBER_OF_C1_IN_D0, NUMBER_OF_C1_IN_D1) != (0,0,0,0):
        if y_C1[i] == 1 and NUMBER_OF_C1_IN_D0 != 0:
            Y_C1.append(1)
            Y_D1.append(0)
            X.append(x[i])
            NUMBER_OF_C1_IN_D0 -= 1
            i += 1
        elif y_C1[i] == 0 and NUMBER_OF_C0_IN_D0 != 0:
            Y_C1.append(0)
            Y_D1.append(0)
            X.append(x[i])
            NUMBER_OF_C0_IN_D0 -= 1
            i += 1
        elif y_C1[i] == 1 and NUMBER_OF_C1_IN_D1 != 0:
            Y_C1.append(1)
            Y_D1.append(1)
            X.append(x[i])
            NUMBER_OF_C1_IN_D1 -= 1
            i += 1
        elif y_C1[i] == 0 and NUMBER_OF_C0_IN_D1 != 0:
            Y_C1.append(0)
            Y_D1.append(1)
            X.append(x[i])
            NUMBER_OF_C0_IN_D1 -= 1
            i += 1
        else:
            i += 1
    X = np.array(X)
    Y_C1 = np.array(Y_C1)
    Y_D1 = np.array(Y_D1)
    
    return train_test_split(X, np.expand_dims(Y_C1, axis=1), np.expand_dims(Y_D1, axis=1), test_size=0.4)

In [4]:
class LogReg(Sequential):
    def __init__(self):
        Sequential.__init__(self)
        self.add(Flatten(input_shape=input_shape))
        self.add(Dense(1, activation='sigmoid'))
        self.compile(loss=keras.losses.binary_crossentropy,
                   optimizer=keras.optimizers.Adam())
        
    def fit(self, dataset):
        X_train, X_test, Y_C1_train, Y_C1_test, Y_D1_train, Y_D1_test = dataset
        
        saver_logreg = keras.callbacks.ModelCheckpoint(filepath='/tmp/logreg.hdf5', monitor='val_loss', verbose=0, save_best_only=True)
        lr_decreaser_logreg = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=10, factor=0.1, epsilon=0.001, verbose=0)

        Sequential.fit(self, X_train, Y_D1_train, validation_data = (X_test, Y_D1_test),
                   verbose=0, batch_size=5000, epochs=2, 
                   callbacks=[saver_logreg, lr_decreaser_logreg])
        
    def rocauc(self, dataset, fractions):
        X_train, X_test, Y_C1_train, Y_C1_test, Y_D1_train, Y_D1_test = dataset
        
        return rocauc(Y_C1_test, D1_problem_to_C1_problem(self.predict(X_test), fractions).flat)

In [28]:
class Cnn(Sequential):
    def __init__(self):
        Sequential.__init__(self)
        self.add(Conv2D(8, kernel_size=(3, 3),
                         activation='relu',
                         input_shape=input_shape))
        self.add(Conv2D(8, (3, 3), activation='relu'))
        self.add(MaxPooling2D(pool_size=(2, 2)))
        self.add(Conv2D(16, (3, 3), activation='relu'))
        self.add(Conv2D(16, (3, 3), activation='relu'))
        self.add(MaxPooling2D(pool_size=(2, 2)))
        self.add(Flatten())
        self.add(Dense(128, activation='relu'))
        self.add(Dense(1, activation='sigmoid'))
        self.compile(loss=keras.losses.binary_crossentropy,
                                optimizer=keras.optimizers.Adam())
        
    def fit(self, dataset, epochs, importance_sampling_classifier=None, importance_sampling_constant=1):
        X_train, X_test, Y_C1_train, Y_C1_test, Y_D1_train, Y_D1_test = dataset
        
        if importance_sampling_classifier is None:
            name = 'cnn_pure'
            Sequential.fit(self, X_train, Y_D1_train, #validation_data = (X_test, Y_D1_test),
                            verbose=1, epochs=epochs)
        else:
            importance_sampling_classifier.load_weights('/tmp/logreg.hdf5')
            
            weights = np.array(importance_sampling_classifier.predict(X_train).flat)
            weights = importance_sampling_constant * np.abs(weights - 0.5)
            weights = (lambda x: np.exp(x) / np.sum(np.exp(x), axis=0))(weights)
            
            for i in range(13): #range(len(X_train) // 1000):
                mask = list(np.random.choice(np.arange(len(X_train)), size=1000, p=weights))
                batch_X = X_train[mask]
                batch_Y = Y_D1_train[mask]
                Sequential.fit(self, batch_X, batch_Y, #validation_data = (X_test, Y_D1_test),
                            verbose=1, epochs=1)
            
            name = 'cnn_aux'
        
        #saver_cnn = keras.callbacks.ModelCheckpoint(filepath='/tmp/'+name, monitor='val_loss', verbose=0, save_best_only=True)
        #lr_decreaser_cnn = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, epsilon=0.001, verbose=0)

        
    def rocauc(self, dataset, fractions):
        X_train, X_test, Y_C1_train, Y_C1_test, Y_D1_train, Y_D1_test = dataset
        
        return rocauc(Y_C1_test, D1_problem_to_C1_problem(self.predict(X_test), fractions).flat)

In [29]:
def experiment(C0_values, fractions, importance_sampling_constant):
    number_of_C0_in_D0, number_of_C0_in_D1, number_of_C1_in_D0, number_of_C1_in_D1 = fractions
    alpha = number_of_C0_in_D0 / (number_of_C0_in_D0 + number_of_C1_in_D0)
    beta = number_of_C0_in_D1 / (number_of_C0_in_D1 + number_of_C1_in_D1)
    
    experiment_logs = {'C0 values': C0_values, 'number of C0 in D0':number_of_C0_in_D0, 'number of C0 in D1':number_of_C0_in_D1, 'number of C1 in D0':number_of_C1_in_D0, 'number of C1 in D1':number_of_C1_in_D1}
    experiment_logs['alpha'] = alpha
    experiment_logs['beta'] = beta
    experiment_logs['importance_sampling_constant'] = importance_sampling_constant
    
    dataset = generate_dataset(C0_values, fractions)
    logreg, cnn_aux, cnn_pure = LogReg(), Cnn(), Cnn()
    
    t0 = time()
    logreg.fit(dataset)
    experiment_logs['logreg time'] = time() - t0
    experiment_logs['logreg rocauc'] = logreg.rocauc(dataset, fractions)
    
    t0 = time()
    cnn_aux.fit(dataset, 1, importance_sampling_classifier=logreg, importance_sampling_constant=importance_sampling_constant)
    experiment_logs['cnn_aux time'] = time() - t0
    experiment_logs['cnn_aux rocauc'] = cnn_aux.rocauc(dataset, fractions)
    #cnn_aux.fit(dataset, 1, auxiliary_classifier=logreg)
    #experiment_logs['cnn_aux rocauc plus epoch'] = cnn_aux.rocauc(dataset, fractions)
    
    t0 = time()
    cnn_pure.fit(dataset, 1)
    experiment_logs['cnn_pure time'] = time() - t0
    experiment_logs['cnn_pure rocauc'] = cnn_pure.rocauc(dataset, fractions)
    #cnn_pure.fit(dataset, 1)
    #experiment_logs['cnn_pure rocauc plus epoch'] = cnn_pure.rocauc(dataset, fractions)
    
    return experiment_logs

In [30]:
importance_sampling_constants = [0.01, 0.1, 1, 10, 100, 1000]

In [31]:
logs = []
for importance_sampling_constant in importance_sampling_constants:
    for i in range(3):
        print('Current experiment: ', i)
        logs.append(
            experiment([0], [30, 450, 31000, 31000], importance_sampling_constant)
            )

Current experiment:  0
Epoch 1/1
1000/1000 [==============================] - 2s - loss: 7.8392     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 7.9074     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 7.6842     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 8.6567     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 8.0350     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 7.8915     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 8.2741     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 7.6842     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 8.2582     
Epoch 1/1
1000/1000 [==============================] - 2s - loss: 8.0190     
Epoch 1/1
1000/1000 [==============================] - 2s - loss: 7.6364     
Epoch 1/1
1000/1000 [==============================] - 2s - loss: 7.9871     
Epoch 1/1
1000/1000 [====================

1000/1000 [==============================] - 1s - loss: 0.7062     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 0.7154     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 0.7048     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 0.7044     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 0.6972     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 0.6876     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 0.6970     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 0.6998     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 0.7005     
Epoch 1/1
37488/37488 [==============================] - 61s - loss: 0.7575    
Current experiment:  2
Epoch 1/1
1000/1000 [==============================] - 2s - loss: 2.2978     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 0.8654     
Epoch 1/1
1000/1000 [============================

1000/1000 [==============================] - 1s - loss: 7.2857     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 7.7958     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 8.1944     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 7.5089     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 7.7639     
Epoch 1/1
37488/37488 [==============================] - 56s - loss: 0.7380    
Current experiment:  0


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: overflow encountered in exp
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in less


Epoch 1/1
1000/1000 [==============================] - 2s - loss: 0.0139     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 1.1921e-07     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 1.1921e-07     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 1.1921e-07     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 1.1921e-07     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 1.1921e-07     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 1.1921e-07     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 1.1921e-07     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 1.1921e-07     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 1.1921e-07     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 1.1921e-07     
Epoch 1/1
1000/1000 [==============================] - 1s - loss: 1.1921e-07     
Epoch 1/1
1000/1000 

In [25]:
for log in logs:
    print(log['logreg rocauc'],log['cnn_aux rocauc'],log['cnn_pure rocauc'])
    # print(log['logreg rocauc'],log['cnn_aux rocauc plus epoch'],log['cnn_pure rocauc plus epoch'])
    print('\n')

0.589474604287 0.775100024694 0.772281581571




In [32]:
pd.DataFrame(logs)

,C0 values,alpha,beta,cnn_aux rocauc,cnn_aux time,cnn_pure rocauc,cnn_pure time,importance_sampling_constant,logreg rocauc,logreg time,number of C0 in D0,number of C0 in D1,number of C1 in D0,number of C1 in D1
0,[0],0.000967,0.014308,0.500000,27.934686,0.545440,75.385020,0.01,0.387129,1.428216,30,450,31000,31000
1,[0],0.000967,0.014308,0.418178,30.856762,0.436780,78.983442,0.01,0.564905,1.486955,30,450,31000,31000
2,[0],0.000967,0.014308,0.652204,29.428717,0.500000,73.759881,0.01,0.693464,1.620913,30,450,31000,31000
3,[0],0.000967,0.014308,0.616173,28.164245,0.896379,76.843398,0.10,0.615444,1.714485,30,450,31000,31000
4,[0],0.000967,0.014308,0.539078,31.313852,0.500000,75.415642,0.10,0.612049,1.794774,30,450,31000,31000
5,[0],0.000967,0.014308,0.652513,28.520525,0.463652,74.071143,0.10,0.674259,1.894963,30,450,31000,31000
6,[0],0.000967,0.014308,0.624387,30.699365,0.841932,72.122681,1.00,0.388551,2.003900,30,450,31000,31000
7,[0],0.000967,0.014308,0.746312,24.829697,0.686297,62.254979,1.00,0.381280,2.142799,30,450,31000,31000
8,[0],0.000967,0.014308,0.685559,24.958680,0.697531,61.758000,1.00,0.565675,2.239764,30,450,31000,31000
9,[0],0.000967,0.014308,0.687537,25.302016,0.710150,60.057472,10.00,0.512336,2.357135,30,450,31000,31000


In [33]:
pd.DataFrame(logs).to_csv('logs_i_s', sep='\t')